In [3]:
import pandas as pd
from price_loaders.tradingview import load_asset_price
from datetime import date

import plotly.express as px
import plotly.graph_objects as go

bases = pd.read_excel("Preços físicos/milho.xlsx", sheet_name="PRAÇAS")
bases['Descrição']

0     Ind. Milho ESALQ/BVMF - Campinas (SP), atacado...
1     Ind. Milho ESALQ/BVMF- Campinas (SP), atacado,...
2     Milho semiduro, Sorriso (MT), atacado, R$/60kg...
3     Milho semiduro, Cascavel (PR), ao produtor, R$...
4     Milho semiduro, Dourados (MS), ao produtor, R$...
5     Milho semiduro, Triângulo Mineiro, atacado, R$...
6     Milho semiduro, Paranaguá, atacado, R$/60kg, à...
7     Milho semiduro, Norte Paraná, ao produtor, R$/...
8     Milho semiduro, Cascavel (PR), atacado, R$/60k...
9     Milho semiduro, Dourados (MS), atacado, R$/60k...
10    Milho semiduro, Norte Paraná, atacado, R$/60kg...
Name: Descrição, dtype: object

In [4]:
baseCode = bases[bases['Descrição'] == "Milho semiduro, Cascavel (PR), ao produtor, R$/60kg, à vista (CDI), sem ICMS"]['Praças'].values[0]
baseCode

'COCCFA'

In [ ]:
commodity = 'Corn'
base = "Milho semiduro, Cascavel (PR), ao produtor, R$/60kg, à vista (CDI), sem ICMS"
futName = 'CCM'
convertUnit = False
expMonth = "1!"
expYear = None
lookback = 5

def basisPlot(commodity, base, futName, convertUnit, expMonth, expYear, lookback):
    fig = go.Figure()

    if commodity == "Corn":
        path = "Preços físicos/milho.xlsx"
        bases = pd.read_excel(path, sheet_name="PRAÇAS")
    else:
        path = "Preços físicos/soja.xlsx"
        bases = pd.read_excel(path, sheet_name="PRAÇAS")
    
    baseCode = bases[bases['Descrição'] == base]['Praças'].values[0]
    del bases

    spotPrices = pd.read_excel(path, sheet_name=baseCode, skiprows=[0,1])
    spotPrices['DRF'] = spotPrices['DRF'].apply(lambda x: pd.Timestamp(x.year, x.month, x.day))
    
    currYear = max(spotPrices['DRF']).year

    if convertUnit:
        dol = pd.read_excel('dolar.xlsx', sheet_name='dol', skiprows=[0,1], header=None)
        dol.columns=["DRF", "R$", "US$", "USD"]

        dol['DRF'] = dol['DRF'].apply(lambda x: pd.Timestamp(x.year, x.month, x.day))

        spotPrices = spotPrices.merge(dol, how = 'left', on='DRF')
        spotPrices['FEC'] = ((spotPrices['FEC']/spotPrices['USD'])/2.36207)*100

        spotPrices.dropna()

    for i in range(0,lookback+1):
        if expMonth == "1!":
            futuresPrices = load_asset_price(f'{futName}{expMonth}', 10000, 'D')
            futuresPrices['DRF'] = futuresPrices['time'].dt.tz_convert("America/Sao_Paulo").apply(lambda x: pd.Timestamp(x.year, x.month, x.day))
            basis = spotPrices.merge(futuresPrices, on = 'DRF', how = 'left')[['DRF', 'FEC', 'close']].ffill()
            basis['basis'] = basis['FEC'] - basis['close']

            yearlySelection = basis[basis['DRF'].dt.year == currYear - i]
            try: yearlySelection = yearlySelection[~((yearlySelection['DRF'].dt.month == 2) & (yearlySelection['DRF'].dt.day == 29))] # # Dropping the 29-Feb of the leap year.
            except: pass
            yearlySelection['DRF'] = yearlySelection['DRF'].apply(lambda x: pd.Timestamp(x.year+i, x.month, x.day))
            if i != 0:
                fig.add_trace(go.Scatter(x=yearlySelection['DRF'], y=yearlySelection['basis'], name=f'{currYear - i}',line=dict(width=1)))
            else: 
                fig.add_trace(go.Scatter(x=yearlySelection['DRF'], y=yearlySelection['basis'], name=f'{currYear - i}',line=dict(width=3)))
        else: 
            futuresPrices = load_asset_price(f'{futName}{expMonth}{expYear}', 10000,'D')

    # Layout monocromático e quadrado
    fig.update_layout(
        title=dict(
            text = f"{base} - {futName}{expMonth}",
            x=0.5,
            font=dict(size=20, color='black', family='Arial')
        ),
        margin=dict(
            b=40,
            l=40,
            r=40,
            t=60
        ),
        width=1200,
        height=800,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(color='black', family='Arial')
    )

    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linewidth=1,
        linecolor='black',
        gridcolor='lightgrey',
        zeroline=False,
        tickfont=dict(size=12, color='black')
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linewidth=1,
        linecolor='black',
        gridcolor='lightgrey',
        zeroline=False,
        tickfont=dict(size=12, color='black')
    )


    fig.show()


basisPlot(commodity, base, futName, convertUnit, expMonth, expYear, lookback)

In [18]:
dol = pd.read_excel('dolar.xlsx', sheet_name='dol', skiprows=[0,1], header=None)
dol.columns=["DRF", "R$", "US$", "USD"]
dol

,DRF,R$,US$,USD
0,2025-07-30,63.51,11.39,5.575944
1,2025-07-29,63.64,11.43,5.567804
2,2025-07-28,63.70,11.38,5.597540
3,2025-07-25,63.74,11.45,5.566812
4,2025-07-24,64.11,11.62,5.517212
...,...,...,...,...
2128,2017-01-06,36.30,NaN,NaN
2129,2017-01-05,36.90,NaN,NaN
2130,2017-01-04,37.25,NaN,NaN
2131,2017-01-03,37.94,NaN,NaN
